In [21]:
import requests



In [22]:
result

<Response [200]>

In [31]:
import pandas as pd
import time

In [34]:
offset = 0
query_params = {"url":"https://europepmc.org/article/MED/29206104",
                "offset":str(offset),
                "limit":"200"}


result = requests.get("https://api.hypothes.is/api/search", params = query_params)

df = pd.json_normalize(result.json()['rows'])

df.head(4)
df_now = df
while len(df_now) == 200:
    time.sleep(1)
    offset += 200
    print("offset: " + str(offset))
    query_params = {"url":"https://europepmc.org/article/MED/29206104",
                "offset":str(offset),
                "limit":"200"}
    result = requests.get("https://api.hypothes.is/api/search", params = query_params)
    df_now = pd.json_normalize(result.json()['rows'])
    df = df.append(df_now)
    

offset: 200
offset: 400
offset: 600
offset: 800
offset: 1000
offset: 1200
offset: 1400
offset: 1600
offset: 1800
offset: 2000
offset: 2200


In [74]:
wikidata_ids = df["text"]

wikidata_ids = wikidata_ids[["entity/Q" in value for value in wikidata_ids]]
wikidata_ids = [value.replace("\n", "") for value in wikidata_ids]


In [75]:
len(set(wikidata_ids))

1093

In [76]:
wikidata_unique_ids = list(set(wikidata_ids))
qid_values = [re.sub(".*Q","wd:Q", qid) for qid in wikidata_unique_ids]


In [106]:
import wikidata2df
import re

qid_values = [re.sub(".*Q","wd:Q", qid) for qid in wikidata_unique_ids]

def get_chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
qid_values_list = get_chunks(qid_values, 400)


list_of = []
i = 0
for qid_values in qid_values_list:
        i+=1
        query = """

        SELECT ?item ?itemLabel ?superclass ?superclassLabel ?type ?typeLabel
        WHERE{
         VALUES ?item {""" + " ".join(qid_values) +"""}
         ?item wdt:P31 ?type.
         ?item wdt:P279 ?superclass.
         SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }

        }
        """
        if i == 1:
            expanded_qid_table = wikidata2df.wikidata2df(query)
        else:
            result_now =  wikidata2df.wikidata2df(query)
            expanded_qid_table = expanded_qid_table.append(result_now)

In [107]:

list_of = []
i = 0
for qid_values in qid_values_list:
        i+=1
        query = """

        SELECT ?item ?itemLabel 
        WHERE{
         VALUES ?item {""" + " ".join(qid_values) +"""}
         SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }

        }
        """
        if i == 1:
            qid_table = wikidata2df.wikidata2df(query)
        else:
            result_now =  wikidata2df.wikidata2df(query)
            qid_table = qid_table.append(result_now)

In [108]:
qid_values_list = get_chunks(qid_values, 400)

In [112]:
qid_table
    

,item,itemLabel
0,Q1553259,insight
1,Q1920566,answer
2,Q560361,draft document
3,Q105495234,3'-end region
4,Q2996394,biological process
...,...,...
288,Q188619,hierarchy
289,Q2944073,myeloid progenitor cell
290,Q11563,number
291,Q105702435,molecular signature
